##BART파인 튜닝 실습: 뉴스 요약
트랜스포머 인코더로 사전 학습(pre-training)된 모델이 BERT, 트랜스 포머 디코더로 사전 학습된 모델이 GPT라면, 인코더-디코더 구조를 유지한 채 사전 학습이 된 모델로 BART와 T5가 있습니다. 이번 실습에서는 BART를 파인 튜닝하여 한국어 뉴스 요약 실습을 진행해 보겠습니다.


###1. 데이터 다운로드
실습을 위해 transformers 패키지와 구글 드라이브로부터 파일을 다운로드 하기 위한 패키지인 gdown
을 설치합니다.

In [1]:
!pip install transformers
!pip install gdown

훈련 데이터와 테스트 데이터를 다운로드 해보겠습니다. 이 데이터는 AI Hub에서 제공하는 데이터 셋입니다.

In [2]:
# 학습 데이터 다운로드
!gdown https://drive.google.com/uc?id=13l621lx2nSnXpFpzh78UUEyds_DAzyn6
# 테스트 데이터 다운로드
!gdown https://drive.google.com/uc?id=10LwhiPlgjOZbtF0Bv5395wYIm23y_QfT

Downloading...
From (original): https://drive.google.com/uc?id=13l621lx2nSnXpFpzh78UUEyds_DAzyn6
From (redirected): https://drive.google.com/uc?id=13l621lx2nSnXpFpzh78UUEyds_DAzyn6&confirm=t&uuid=d2c0309d-1c47-42c9-b393-1632e5cd1e96
To: /content/summ_train.json
100% 1.16G/1.16G [00:14<00:00, 80.7MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=10LwhiPlgjOZbtF0Bv5395wYIm23y_QfT
From (redirected): https://drive.google.com/uc?id=10LwhiPlgjOZbtF0Bv5395wYIm23y_QfT&confirm=t&uuid=fec57aa9-f3d5-47f9-992c-a7587792109b
To: /content/summ_test.json
100% 147M/147M [00:01<00:00, 97.7MB/s]


각각 summ_train.json, summ_test.json 파일이 다운로드 됩니다. 판다스의 데이터프레임을 이용하여
해당 파일들을 로드해보겠습니다. 실습에서의 학습 시간을 절약하기 위해서 전체 데이터셋 중에서 학습
데이터는 4 만개, 테스트 데이터는 5000 개만 사용하겠습니다. 물론, 실제로는 더 많은 데이터셋을 학습
할 수록 성능이 더 좋기 때문에 현업의 서비스에서 사용하실 생각이라면 제공하는 모든 데이터를 활용하
시기 바랍니다.

In [3]:
import pandas as pd

data_train_path = 'summ_train.json'
train_df = pd.read_json(data_train_path)
train_df = train_df.dropna()
train_df = train_df[:4000]
print('학습 데이터 개수: ', len(train_df))

data_test_path = 'summ_test.json'
test_df = pd.read_json(data_test_path)
test_df = test_df.dropna()
test_df = test_df[:5000]
print('테스트 데이터 개수: ', len(test_df))

학습 데이터 개수:  4000
테스트 데이터 개수:  5000


AI Hub에서 제공하는 이 데이터는 다소 복작한 구조를 갖고 있습니다. 학습데이터와 테스트 데이터 각각 상위 5개만 출력해 봅시다.

In [4]:
train_df.head()

,name,delivery_date,documents
0,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741778', 'category': '종합', 'media_t..."
1,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741792', 'category': '종합', 'media_t..."
2,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741793', 'category': '스포츠', 'media_..."
3,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741794', 'category': '정치', 'media_t..."
4,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741797', 'category': '종합', 'media_t..."


In [5]:
test_df.head()

,name,delivery_date,documents
0,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340626877', 'category': '정치', 'media_t..."
1,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340626896', 'category': '종합', 'media_t..."
2,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340626904', 'category': 'IT,과학', 'medi..."
3,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340627450', 'category': '사회', 'media_t..."
4,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340627465', 'category': '경제', 'media_t..."


BART 학습에 사용하기 위해서는 ‘요약하기 전의 원문’ 과’ 요약문’ 이 두 가지만 있으면 됩니다. 참고로 이 두 개의 텍스트는 위의 데이터프레임에서 ‘documents’ 열에 저장되어져 있습니다. 이를 파싱
하여 원문은’article_original’ 열에, 그리고 요약문은 ‘abstractive’ 열에 저장하는 아래의 전처리 함수
preprocess_data() 를 사용하여 새로운 데이터프레임’train_data’ 와’test_data’ 를 얻어보겠습니다.

In [6]:
def preprocess_data(data):
  outs = []
  for doc in data['documents']:
    line = []
    line.append(doc['media_name'])
    line.append(doc['id'])
    para = []
    for sent in doc ['text']:
      for s in sent:
        para.append(s['sentence'])
    line.append(para)
    line.append(doc['abstractive'][0])
    line.append(doc['extractive'])
    a = doc['extractive']
    if a[0] == None or a[1] == None or a[2] == None:
      continue
    outs.append(line)

  outs_df = pd.DataFrame(outs)
  outs_df.columns = ['media', 'id', 'article_original', 'abstractive', 'extractive']
  return outs_df

전처리 후의 학습 데이터를 출력해봅시다.

In [7]:
# 원문과 요약문을 각각 'article_original'와 'abstractive'열에 저장.
train_data = preprocess_data(train_df)
train_data.head()

,media,id,article_original,abstractive,extractive
0,광양신문,290741778,"[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,"[2, 3, 10]"
1,광양신문,290741792,"[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,"[2, 4, 11]"
2,광양신문,290741793,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...",전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,"[3, 5, 7]"
3,광양신문,290741794,"[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작...","[2, 3, 4]"
4,광양신문,290741797,"[홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항, 이성훈 sinawi@han...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...,"[3, 7, 4]"


In [8]:
test_data = preprocess_data(test_df)
test_data.head()

,media,id,article_original,abstractive,extractive
0,한국경제,340626877,"[[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, ...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,"[0, 1, 3]"
1,한국경제,340626896,"[] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다., 바른미래당이 8일 ...",8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...,"[2, 1, 6]"
2,한국경제,340626904,"[[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀., 지난 3일 한국이 세계 최초...",지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...,"[1, 5, 8]"
3,한국경제,340627450,[] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...,"[0, 1, 2]"
4,한국경제,340627465,"[[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...,"[1, 3, 4]"


원문과 요약문이 각각’article_original’ 와’abstractive’ 열에 저장된 데이터프레임을 얻었습니다.
’extractive’ 열의 경우 지금 여기서 하고자하는 생성 요약을 위한 데이터셋이 아니라 BERTSum 과 같
은 추출 요약 모델을 위한 레이블이므로 여기서는 사용하지 않습니다. train_data 의 첫번째 샘플의
article_original 열의 값을 출력해보겠습니다.

In [9]:
train_data['article_original'].loc[0]

['ha당 조사료 400만원…작물별 차등 지원',
 '이성훈 sinawi@hanmail.net',
 '전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다.',
 '쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다.',
 '올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다.',
 '지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다.',
 '지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다.',
 '농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다.',
 'ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다.',
 '벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다.',
 '논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다.',
 '전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다.',
 '최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.']

위의 결과를 보면 현재 원문이 저장된 article_original 의 경우에는 각 문장을 원소로 하는 파이썬의 리스
트 형태로 저장되어져 있어 이를 하나의 본문으로 저장하여’news’ 열에 저장하고, train_data 의 첫번째
샘플의 news 열의 값을 출력해보겠습니다.

In [10]:
train_data['news'] = train_data['article_original'].apply(lambda x : ' '.join(x) )
test_data['news'] = test_data['article_original'].apply(lambda x : ' '.join(x) )
train_data['news'].loc[0]

'ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.'

In [11]:
train_data[['news', 'abstractive']].head()

,news,abstractive
0,ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,"8억 투입, 고소천사벽화·자산마을에 색채 입혀 이성훈 sinawi@hanmail.n...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,전남드래곤즈 해맞이 다짐…선수 영입 활발 이성훈 sinawi@hanmail.net ...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,"11~24일, 매실·감·참다래 등 지역특화작목 이성훈 sinawi@hanmail.n...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작..."
4,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 이성훈 sinawi@hanma...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...


###2. 정수 인코딩을 위한 Dataset생성


In [12]:
import tensorflow as tf
from transformers import TFBartForConditionalGeneration, BartTokenizerFast
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import CosineDecay
import numpy as np
from tqdm import tqdm

BART모델에 사용될 입력 데이터와 레이블을 준비하는 데이터셋 클래스를 작성합니다.

In [13]:
class KoBARTSummaryDataset(tf.keras.utils.Sequence):
  def __init__(self, df, tokenizer, max_len, batch_size, ignore_index=-100):
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.docs = df
    self.batch_size = batch_size
    self.ignore_index = ignore_index
    self.indices = list(range(len(self.docs)))
  def __len__(self):
    return len(self.indices) // self.batch_size

  def __getitem__(self, idx):
    if idx >= len(self):
      raise IndexError("Index out of range")

    batch_indices = self.indices[idx * self.batch_size : (idx + 1) * self.batch_size]
    batch = self.docs.iloc[batch_indices]

    input_ids = []
    decoder_input_ids = []
    labels = []
    for _, instance in batch.iterrows():
      # 'news' 열의 텍스트를 정수 인코딩 하여 입력에 해당하는 'input_ids' 생성
      encoded_input = self.tokenizer.encode(instance['news'], max_length=self.max_len,
                                            padding='max_length', truncation=True)
      input_ids.append(encoded_input)

      # 'abstractive' 열의 텍스트를 정수 인코딩 하여 레이블 생성
      encoded_label = self.tokenizer.encode(instance['abstractive'],
                                            max_length=self.max_len, padding='max_length',
                                            truncation=True)
      decoder_input = [self.tokenizer.eos_token_id] + encoded_label[:-1]
      decoder_input_ids.append(decoder_input)

      # 레이블에 -100을 이용하여 패딩을 적용
      label = encoded_label + [self.ignore_index] * (self.max_len - len(encoded_label))
      labels.append(label)
    return {'input_ids': np.array(input_ids),
            'decoder_input_ids': np.array(decoder_input_ids),
            'labels': np.array(labels)}

  def on_epoch_end(self):
    np.random.shuffle(self.indices)

크게 아래의 순서로 전처리를 수행합니다.
KoBARTSummaryDataset 클래스는 데이터를 배치 형태로 제공하는 역할을 합니다.
init 메서드는 클래스 인스턴스가 생성될 때 호출됩니다. 이 메서드에서는 데이터셋 (df), 토크나이저
(tokenizer), 입력 시퀀스의 최대 길이 (max_len), 배치 크기 (batch_size), 그리고 학습 시 무시할 인덱스
(ignore_index) 를 초기화합니다.
getitem 메서드는 주어진 인덱스에 해당하는 배치 데이터를 반환하는 데 사용됩니다. 이 메서드는 매우
중요한 역할을 합니다. 먼저, 지정된 인덱스를 기반으로 해당 배치에 포함될 데이터 샘플의 인덱스를 계
산합니다. 그런 다음, 이 인덱스를 사용하여 DataFrame 에서 해당하는 데이터 샘플들을 추출합니다. 추
출된 데이터는 뉴스 기사와 요약문으로 구성됩니다.
각 뉴스 기사는 토크나이저를 사용해 토큰으로 변환되며, 이 토큰들은 input_ids 리스트에 추가됩니다.
이 과정에서 뉴스 기사의 길이가 max_len 을 초과하는 경우, 텍스트는 잘리게 되고, 길이가 부족할 경우
에는 패딩이 추가됩니다. 요약문은 라벨 데이터로 사용되며, 마찬가지로 토크나이저를 통해 토큰으로 변
환됩니다. 이 토큰들은 decoder_input_ids 와 labels 리스트로 처리됩니다.
decoder_input_ids 는 디코더의 입력으로 사용되며, 여기에는 시작 토큰이 추가되고 마지막 토큰이 제거
된 형태로 구성됩니다. labels 는 모델이 예측해야 할 정답 시퀀스를 나타내며, 이 시퀀스가 max_len 보다
짧으면 나머지 부분은 ignore_index 로 채워집니다. 이 ignore_index 값은 기본적으로 ‐100 으로 설정되
어 있으며, 이는 패딩된 부분이 학습 과정에서 무시되도록 합니다.
마지막으로, 이러한 데이터들은 모두 NumPy 배열로 변환되어 딕셔너리 형태로 반환됩니다. 반환되는 딕
셔너리에는 input_ids, decoder_input_ids, 그리고 labels 가 포함됩니다.

### 3. 모델 클래스 선언이제 모델을

선언해보겠습니다. KoBARTConditionalGeneration 클래스는 텍스트 생성을 위해 BART 모
델을 사용하며, 한국어 텍스트에 특화된 KoBART 모델을 사용합니다.

In [14]:
model = TFBartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v1',
                                                       from_pt=True)
tokenizer = BartTokenizerFast.from_pretrained('gogamza/kobart-base-v1')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForConditionalGeneration: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
- This IS expected if you are initializing TFBartForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, yo

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'BartTokenizerFast'.


### 4. 데이터로더 변환
이제 학습을 위한 하이퍼 파라미터를 설정합니다.

In [15]:
batch_size = 2
max_len = 512
lr = 3e-5
max_epochs = 2
warmup_ratio = 0.1

배치 크기는 2, 모델의 입력으로 사용할 데이터의 최대 길이는 512, 학습률 (learning rate) 는 3e‐5 입니다.
이는 0.00003 을 의미합니다. 학습 횟수에 해당하는 max_epochs 의 값은 2, warmup_ratio = 0.1 는 학습
초기에 학습률을 점진적으로 증가시키는 비율을 나타냅니다. 전체 학습 과정의 10% 동안 학습률이 점진
적으로 증가합니다.

In [16]:
train_dataset = KoBARTSummaryDataset(train_data, tokenizer, max_len=max_len,
batch_size=batch_size)
test_dataset = KoBARTSummaryDataset(test_data, tokenizer, max_len=max_len,
batch_size=batch_size)

total_steps = len(train_dataset) * max_epochs
warmup_steps = int(total_steps * warmup_ratio)
lr_schedule = CosineDecay(initial_learning_rate=lr, decay_steps=total_steps)
optimizer = Adam(learning_rate=lr_schedule)

total_steps = len(train_dataset)* max_epochs는 전체 학습 과정에서 총 몇 번의 업데
이트 (스텝) 가 이루어질지를 계산합니다. 이는 데이터셋의 배치 수와 에포크 수를 곱한 값입니다.

예를 들어, 데이터셋에 100 개의 배치가 있고, 2 번의 에포크가 있다면 총 200 번의 업데이트가 이루어집니다.

warmup_steps = int(total_steps * warmup_ratio)는 학습 초기 단계에서 학습률이 점진적으로 증가하는 스텝 수를 계산합니다.

예를 들어, 총 200 스텝 중에서 10% 인 20 스텝 동안 학습률이 점진적으로 증가하게 됩니다.

lr_schedule = CosineDecay(initial_learning_rate=lr, decay_steps=total_steps)는 Cosine Decay 스케줄을 사용하여 학습률이 학습과정 동안 점진적 으로 감소하도록 설정합니다.학습률은 초기에는 lr로 설정되고, 전체 학습 스텝 동안 점진적으로 감소합니
다.

마지막으로, optimizer = Adam(learning_rate=lr_schedule)는 Adam 최적화 알고리즘을 설정합니다. 학습률은 앞에서 정의한 lr_schedule에 따라 변합니다. Adam 최적화 알고리즘은 모멘텀을 이용해 학습 속도를 빠르게 하고, 안정적인 수렴을 가능하게 합니다.
이 코드는 학습 초기 단계에서는 학습률을 점진적으로 증가시키고, 전체 학습 과정에서는 Cosine Decay 스케줄에 따라 학습률을 감소시키면서 모델을 최적화하도록 설정된 것입니다.

###5. 데이터 확인
정수 인코딩과 패딩이라는 전처리 전, 후 어떤 차이가 있는지 살펴봅시다. 예를 들어서 정수 인코딩 전의
학습 데이터의 첫번째 샘플의 원문은 다음과 같았습니다.

In [17]:
print('첫 샘플의 원문')
print("--"*50)
train_data['news'].loc[0]

첫 샘플의 원문
----------------------------------------------------------------------------------------------------


'ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.'

이를 tokenizer.encode() 를 통해 정수 인코딩 후에 최대 길이 512 를 맞추기 위해서 패딩한 결과는 다음과
같습니다.

In [18]:
print('첫 샘플의 원문 정수 인코딩 및 패딩 결과')
print("--"*50)
train_dataset[0]['input_ids'][0]


첫 샘플의 원문 정수 인코딩 및 패딩 결과
----------------------------------------------------------------------------------------------------


array([21582,   296,  9770, 14666, 10386, 14136, 16266, 14476, 12061,
       10675, 10872, 14165, 10001, 14423, 19168, 13758, 18482, 14885,
         296,   318,   304,   263,   303, 15195,   308,   296, 17761,
         245, 26818, 26102,  9506, 14973, 18193, 24873, 24777, 27841,
       25486, 14185, 26379, 15358, 14049, 18193, 14871, 17074, 14902,
       15055, 14634, 17613, 19754, 18193, 14871, 17074, 16245, 19609,
       10443, 14291, 15912, 14396, 11786, 19609, 15410, 14031, 10386,
       12061, 10675, 14303, 20054, 14048, 14355, 14212, 15049, 14291,
       14553, 19609, 15231, 16864, 16365, 15828, 26030, 19560, 16617,
       14130, 14516, 16309, 12024, 14396, 14355, 14212, 10675, 19696,
       14491, 20087, 12005, 14770, 21598, 26407, 12024, 14383, 16167,
         236, 12037, 16248, 14195, 25292, 26407, 15888, 20066, 29246,
       12332, 21405, 16053,   243, 14044, 20169, 19304, 15615, 14423,
       14573, 24884, 19961, 11211, 18193, 19819, 12333, 10955, 14842,
       15141, 14497,

In [19]:
print('정수 인코딩 및 패딩 후의 길이')
print('****'*20)
len(train_dataset[0]['input_ids'][0])

정수 인코딩 및 패딩 후의 길이
********************************************************************************


512

정수 인코딩이 되면서 텍스트가 정수 시퀀스로 변경되었을 뿐만 아
니라 뒤에 숫자 3 이 연속해서 등장합니다. 최대 길이를 512 로 설정하였으므로 길이를 512 로 맞추기 위
해서 패딩된 것으로 SKT 의 KoBART 는 패딩 토큰 <PAD>가 숫자 3 입니다. 이번에는 원문이 아니라 요약
문을 가지고 확인해봅시다.

In [20]:
print('첫 샘플의 요약문')
print('--'*50)
train_data['abstractive'].loc[0]

첫 샘플의 요약문
----------------------------------------------------------------------------------------------------


"전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 ."

이를 tokenizer.encode() 를 통해 정수 인코딩 후에 최대 길이 512 를 맞추기 위해서 패딩한 결과는 다음과
같습니다.

In [21]:
print('첫 샘플의 요약문 정수 인코딩 및 패딩 결과')
print("--"*50)
train_dataset[0]['decoder_input_ids'][0]

첫 샘플의 요약문 정수 인코딩 및 패딩 결과
----------------------------------------------------------------------------------------------------


array([    1, 26102,  9506, 14973, 18193, 24873, 24777, 27841, 25486,
       14185, 26379, 19609, 10443, 14291, 15912, 14396, 11786, 19609,
       15410, 14031, 10386, 12061, 10675, 14303, 20054, 14048, 14355,
       14212, 15049, 14291, 14553, 19609, 15231, 16864, 16365, 15828,
       26030, 19560, 14063, 11495, 14871, 17074, 12147, 15127, 17489,
       15358, 15272, 14432, 14834, 15271, 15243, 15869, 15450, 15364,
       14497, 12332, 16516, 12332, 23891, 10586,  9879, 17982, 18290,
       15453, 17210,  9754, 17546,     3,     3,     3,     3,     3,
           3,     3,     3,     3,     3,     3,     3,     3,     3,
           3,     3,     3,     3,     3,     3,     3,     3,     3,
           3,     3,     3,     3,     3,     3,     3,     3,     3,
           3,     3,     3,     3,     3,     3,     3,     3,     3,
           3,     3,     3,     3,     3,     3,     3,     3,     3,
           3,     3,     3,     3,     3,     3,     3,     3,     3,
           3,     3,

In [22]:
print('정수 인코딩 및 패딩 후의 길이')
print('--' * 50)
len(train_dataset[0]['decoder_input_ids'][0])

정수 인코딩 및 패딩 후의 길이
----------------------------------------------------------------------------------------------------


512

이번에는 레이블을 정수 인코딩 후 다시 텍스트로 복원한 결과를 확인해봅시다.

In [23]:
# -100의 값은 tokenizer_decode하면 에러나므로 임시로 0으로 변경 후 출력
test_array = train_dataset[0]['labels'][0]
test_array[test_array == -100] = 0
print('첫 샘플의 요약문 레이블')
tokenizer.decode(test_array)

첫 샘플의 요약문 레이블


"전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 .<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

### 6. 학습
이제 모델을 학습해봅시다. 딥러닝 모델을 학습시키고 검증 데이터에서의 손실을 평가하여, 최적의 손실
값을 가진 모델을 저장합니다. 각 epoch 마다 학습과 평가를 수행하며, 평가 손실이 이전보다 낮으면 모
델의 상태를 저장합니다. 이는 성능이 좋은 모델을 저장하는 것이 목적으로 추후 다시 로드하여 활용할 수
있습니다.


In [25]:
# best_loss는 초기값으로 float('inf')로 초기화
best_loss = float('inf')

for epoch in range(max_epochs):
  print(f'에포크 {epoch + 1}/ {max_epochs}')

  # 에포크 동안 누적됭 총 손실을 저장할 변수
  total_train_loss = 0.0

  # 데이터를 배치 크기 만큼 꺼내서 각 배치에 대해 모델을 학습
  for batch in tqdm(train_dataset, total=len(train_dataset), desc="훈련 중"):
    with tf.GradientTape() as tape:
      outputs = model(batch, training=True)
      loss = outputs.loss

    #모델의 손실(outputs.loss) 값으로 부터 연전파를 수행하여 파라키터를 업데이트.
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    total_train_loss += tf.reduce_mean(loss).numpy()

  #  '''한 에포크 동안 모든 배치에서 발생한 손실을 배치 수로 나누어, 한 에포크 동안 배치 크기만큼의
  #     데이터를 넣었을 때 발생한 평균 손실을 계산'''
  avg_train_loss = total_train_loss / len(train_dataset)
  print(f'훈련 손실: {avg_train_loss:.4f}')

  # 평가 단계 전체
  total_val_loss = 0.0 # Initialize total_val_loss
  for batch in tqdm(test_dataset, total=len(test_dataset), desc="검증 중"):
    outputs = model(batch, training=False)
    total_val_loss += tf.reduce_mean(outputs.loss).numpy()

  avg_val_loss = total_val_loss / len(test_dataset)
  print(f'검증 손실: {avg_val_loss:.4f}')

  # 최고 성능 모델 저장
  if avg_val_loss < best_loss:
    best_loss = avg_val_loss
    # model.save_weights('best_model.weights.h5')
    # tf.saved_model.save(model, 'best_model')
    model.save_pretrained('best_model')
    print(f'검증 손실이 {best_loss:.4f}로 계선. 체크포인트로 저장')

  # 에포크 종료시 데이터 사용
  train_dataset.on_epoch_end()
  test_dataset.on_epoch_end()
print("훈련이 완료되었습니다.")

에포크 1/ 2


훈련 중: 100%|██████████| 2000/2000 [1:03:44<00:00,  1.91s/it]


훈련 손실: 1.2039


검증 중: 100%|██████████| 2499/2499 [14:20<00:00,  2.90it/s]
/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:407: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'forced_eos_token_id': 1}
  warnings.warn(


검증 손실: 1.7276
검증 손실이 1.7276로 계선. 체크포인트로 저장
에포크 2/ 2


훈련 중: 100%|██████████| 2000/2000 [1:03:43<00:00,  1.91s/it]


훈련 손실: 1.0783


검증 중: 100%|██████████| 2499/2499 [14:26<00:00,  2.88it/s]


검증 손실: 1.7253
검증 손실이 1.7253로 계선. 체크포인트로 저장
훈련이 완료되었습니다.


학습은 max_epochs로 지정된 횟수만큼 반복됩니다. best_loss는 초기값으로 float('inf'),
즉 무한대로 초기화합니다.
for 문의 시작 부분에서는 각 에포크에 대해 현재 진행 중인 에포크 번호와 총 에포크 수를 출력합니다.
훈련 단계에서는 먼저 total_train_loss를 0 으로 초기화합니다. 이 변수는 에포크 동안 누적된 총
손실을 저장하기 위한 것입니다. 훈련 데이터셋 (train_dataset) 을 배치 단위로 반복하면서 각 배치
에 대해 모델을 학습시킵니다. with tf.GradientTape()as tape: 문을 사용하여 손실에 대한
그래디언트를 계산하기 위한 준비를 합니다.
모델에 배치 데이터를 입력할 때, outputs = model(batch, training=True)을 사용하여
모델을 학습 모드로 설정합니다. 이 과정에서 모델은 배치 크기만큼의 데이터를 사용해 예측을 수
행하고, 손실 값을 계산합니다. 계산된 손실은 outputs.loss로 접근할 수 있으며, 이 손실 값을
total_train_loss에 더합니다. tf.reduce_mean(loss).numpy()를 사용해 배치 손실의 평
균을 구합니다.
tape.gradient(loss, model.trainable_variables)를 사용해 손실에 대한 모델의 가중
치 (파라미터) 의 기울기 (그래디언트) 를 계산합니다. 그런 다음, optimizer.apply_gradients
(zip(gradients, model.trainable_variables))을 통해 모델의 가중치를 업데이트합니
다.
모 든 배 치 에 대 해 학 습 이 완 료 되 면, avg_train_loss = total_train_loss / len(
train_dataset)를 통해 에포크 동안의 배치 크기만큼의 데이터를 넣었을 때의 평균 훈련 손실을 계
산하여 출력합니다.
평가 단계에서는 훈련된 모델을 검증 데이터셋 (test_dataset) 을 사용해 평가합니다. 모델은
outputs = model(batch, training=False)로 설정되어 평가 모드로 작동하며, 각 배치에서
계산된 손실은 total_val_loss에 더해지며, avg_val_loss = total_val_loss / len(
test_dataset)를 통해 전체 검증 데이터셋에 대한 배치 크기만큼의 데이터를 넣었을 때의 평균 손실
을 계산합니다. 이 평균 검증 손실을 출력하여 모델의 성능을 평가합니다.
모델의 성능을 평가한 후, 만약 현재의 avg_val_loss가 이전에 기록된 best_loss보다 낮다면, 현
재 모델이 “최고 성능” 의 모델입니다. 이 경우 best_loss를 현재의 avg_val_loss로 업데이트하
고, model.save_pretrained('model')을 사용하여 모델을 저장합니다. 이 저장된 모델은 가장
낮은 검증 손실을 기록한 최상의 모델로, 나중에 사용할 수 있도록 체크포인트를 저장합니다.
각에포크가끝날때마다train_dataset.on_epoch_end()와test_dataset.on_epoch_end
()를 호출하여 데이터셋의 순서를 무작위로 섞습니다. 모델이 다음 에포크에서 데이터를 다른 순서로 학
습하게 하여, 데이터 순서에 따른 편향을 방지하고 일반화 능력을 향상시키는 데 기여합니다.

###7. 로드 및 요약문 생성
학습이 정상적으로 끝났다면 현재 경로에 검증 데이터에 대한 손실이 가장 적었을 때의 모델인 model 이
저장됩니다. 이제 해당 모델을 로드하여 임의의 데이터에 대해서 평가해봅시다.

In [26]:
# 모델 로드
loaded_model = TFBartForConditionalGeneration.from_pretrained('best_model')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at best_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


임의의 입력에 대해서 요약문을 생성하는 함수를 만듭니다.

In [27]:
def summarize(text, model, tokenizer, max_length=300):
  # 원 문 을 토 큰 화
  inputs = tokenizer.encode(text, return_tensors="tf", max_length=512, truncation=True)

  # 요 약 문 생 성 (greedy decoding 사 용)
  summary_ids = model.generate(inputs, max_length=max_length, num_beams=7, repetition_penalty=2.0)

  # 토 큰 을 문 자 열 로 변 환
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return summary

inputs = tokenizer.encode(text, return_tensors="tf", max_length=512,truncation=True)는 원문 텍스트를 토큰화합니다. 여기서 tokenizer.encode는 텍스트
를 모델이 이해할 수 있는 토큰의 형태로 변환합니다. return_tensors="tf"는 반환된 토큰이
TensorFlow 텐서로 변환되도록 하며, max_length=512는 입력 토큰의 최대 길이를 512 로 제한합니
다. truncation=True는 텍스트가 최대 길이를 초과할 경우 잘라내도록 설정합니다.
summary_ids = model.generate(inputs, max_length=max_length, num_beams
=7, repetition_penalty=2.0)는 모델이 토큰화된 입력을 사용해 요약문을 생성합니다.
generate 함수는 텍스트 생성 시 사용되며, 여기서는 max_length=max_length로 요약문의 최대
길이를 지정합니다. num_beams=7는 빔 서치 (beam search) 전략을 사용해 7 개의 경로를 탐색하며,
repetition_penalty=2.0는 반복되는 단어를 억제하는 역할을 합니다.
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
는 모델이 생성한 요약문 토큰을 다시 문자열로 변환합니다. skip_special_tokens=True는 특수
토큰 (예: <s>, </s>) 을 제거하여 최종 요약문이 깨끗하게 나오도록 합니다.
마지막으로, return summary에서 생성된 요약문을 반환합니다. 이 함수는 입력된 긴 텍스트를 간결
하게 요약된 텍스트로 변환하는 역할을 합니다.
임의로 테스트 데이터의 25 번 샘플을 text 라는 변수에 저장해봅시다.

In [28]:
text = test_data.loc[25]['news']
print(text)

배우 배수지가 매니지먼트 숲과 전속계약을 체결했다. 수지는 8일 자신의 인스타그램에 '데뷔 때 부터 함께해온 소속사 JYP와 계약기간을 마치고 오늘부터 새로운 소속사 매니지먼트 숲과 함께 하게 되었다'고 밝혔다. 이어 수지는 '연습생으로 시작해서, 데뷔하고 9년의 시간이 흐른 지금까지, JYP와 함께했던 여러 영광의 순간들이 스쳐지나간다'면서 '9년 동안 항상 옆에서 서포트 해주셨던 JYP 모든 직원분들께 진심으로 감사드린다'고 인사를 잊지 않았다. 2010년 걸그룹 '미쓰에이'로 데뷔한 배수지는 2011년 KBS2 드라마 '드림하이'로 첫 연기 활동을 시작했다. 2012년 영화 '건축학개론'을 통해 스크린 데뷔를 한 뒤 가수 활동과 연기 활동을 꾸준히 병행해 오고 있다. 매니지먼트 숲 관계자는 '배우 배수지의 장점과 매력을 극대화할 수 있는 작품 선택부터 국내외 활동, 가수로서의 솔로 활동까지 활발하게 이루어질 수 있도록 지원할 예정이다'고 전했다. 특히 올해는 작품을 통해 연기자 배수지로 대중들과 만날 예정이다. 현재 촬영 중인 SBS 드라마 '배가본드'는 민항 여객기 추락 사고에 연루된 한 남자가 은폐된 진실 속에서 찾아낸 거대한 국가 비리를 파헤치게 되는 과정을 담은 이야기다. 배수지는 국정원 블랙요원 고해리 역으로 출연하며, 뒤이어 영화 '백두산'에도 합류한다. 매니지먼트 숲은 공유, 공효진, 김재욱, 서현진, 이천희, 전도연, 정유미, 남지현, 최우식, 유민규, 이재준, 정가람, 전소니 등 소속되어 있다.


학습한 BART 모델의 입력으로 사용하고 최종적으로 얻은 요약문을 출력해봅시다.

In [29]:
summary = summarize(text, model, tokenizer)
print(summary)

배우 배수지가 매니지먼트 숲과 전속계약을 체결했다. 수지는 8일 자신의 인스타그램에 '데뷔 때 부터 함께해온 소속사 JYP와 계약기간을 마치고 오늘부터 새로운 소속사 매니지먼트 숲과 함께 하게 되었다'고 밝혔다.                                                                                                                                                                                                                                                     
